# 📘 NLP 基础：文本预处理知识点梳理

> **基于文件**：`247_文本预处理.ipynb`
> **核心目标**：将人类读得懂的“文本”，转换为计算机算得动的“数字”。

---

## 1. 核心流程总览

自然语言处理（NLP）的第一步永远是预处理。想象你要教一个只认识数字的外星人读地球的书，流程如下：

1.  **原始文本**： `"I love AI"`
    ↓ *(Tokenization 切分)*
2.  **词元 (Tokens)**： `["I", "love", "AI"]`
    ↓ *(Vocabulary 对照字典)*
3.  **词表映射**： `{"I": 10, "love": 20, "AI": 30}`
    ↓ *(Corpus 转换)*
4.  **最终输出 (Corpus)**： `[10, 20, 30]`

---

## 2. 知识点详解

### 知识点 1：读取数据集 (Reading the Dataset)
* **概念**：将原始文本文件加载到内存中。
* **通俗解释**：就像要把大象装冰箱，第一步得先找到大象。代码首先要把书（如《时光机器》）的内容读取成字符串。
* **简单例子**：
    假设我们的书里只有一句话：
    > *"The time machine is fun."*
    这就是我们的**原始数据**。

### 知识点 2：词元化 (Tokenization)
* **概念**：将长文本“切”成最小的处理单位。
* **通俗解释**：计算机处理长句子很累，它更喜欢处理一个个小积木。这些小积木就叫 **Token（词元）**。
* **代码对应**：文件中定义了 `tokenize` 函数。
* **常见切法**：
    * **按单词切 (Word-level)**：`['The', 'time', 'machine', 'is', 'fun']` （最常用）
    * **按字符切 (Char-level)**：`['T', 'h', 'e', ' ', 't', 'i', ...]`

### 知识点 3：构建词表 (Vocabulary)
* **概念**：建立一个“单词 <-> 数字索引”的映射表。
* **通俗解释**：给每个词发一个唯一的“身份证号”。如果不发身份证，计算机不知道 'apple' 和 'banana' 有什么区别。
* **代码对应**：文件中定义了 `Vocab` 类。
* **核心逻辑**：
    1.  **统计频率**：数数每个词出现了几次。
    2.  **过滤**：去掉出现次数太少（< min_freq）的生僻词，减少计算量。
    3.  **特殊标记**：
        * `<unk>` (unknown)：遇到没见过的词，统统叫这个代号。
        * `<pad>`：补齐长度用的占位符。
        * `<bos>` / `<eos>`：句子的开始 (Begin) 和结束 (End)。
* **简单例子**：
    ```python
    # 词表 (Vocab)
    {
      "The": 0,
      "time": 1,
      "machine": 2,
      "<unk>": 3
    }
    ```

### 知识点 4：语料库转换 (Corpus Construction)
* **概念**：将文本序列转换为索引序列。
* **通俗解释**：这是最后一步，拿着“词表”这个字典，把整本书翻译成一串数字。这串数字才是模型真正吃进去的数据。
* **代码对应**：代码中通常通过列表推导式 `[vocab[token] for token in tokens]` 实现。
* **简单例子**：
    * **输入**：`"The time machine"`
    * **查表转换**：`[0, 1, 2]`
    * **输入（含生词）**：`"The apple"` （假设 apple 不在词表中）
    * **查表转换**：`[0, 3]` （3 代表 unknown）

---

## 3. 总结

这份代码展示了深度学习 NLP 任务的**第一块基石**。

| 步骤 | 输入 (Input) | 输出 (Output) | 作用 |
| :--- | :--- | :--- | :--- |
| **1. 读取** | 文本文件 | 字符串 | 获取原始素材 |
| **2. 词元化** | 字符串 | Token 列表 | 把句子切碎成词 |
| **3. 建词表** | Token 列表 | 字典 (Map) | 给每个词分配 ID |
| **4. 转语料** | Token 列表 | 索引列表 (Int List) | **数字化**，供模型计算 |

# 1. 文本预处理

In [ ]:
import collections  
# [语法]: 导入 Python 标准库 collections (容器数据类型)
# [作用]: 
# 1. 在 NLP 中，我们最常用的是 collections.Counter 这个工具。
# 2. 它的作用是“数数”。比如给你一篇文章，它能自动统计出每个单词出现了多少次（词频统计）。
# 3. 统计完词频后，我们才能构建“词表 (Vocab)”，把那些出现次数太少的生僻词过滤掉。

import re  
# [语法]: 导入 Python 标准库 re (Regular Expression 正则表达式)
# [作用]: 
# 1. 它是文本清洗的神器。
# 2. 原始文本里有很多我们要去掉的脏数据，比如标点符号 (,, !, ?)、特殊字符 (@, #) 或者多余的空格。
# 3. 你的代码中用它来把所有“非字母”的字符替换成空格，只保留纯英文单词。

① 将数据集读取到由文本行组成的列表中。

In [5]:
import os  # [语法]: 导入标准库 os (Operating System)。 [作用]: 用于检查文件是否存在、处理文件路径。
import re  # [语法]: 导入标准库 re (Regular Expression)。 [作用]: 用于正则表达式匹配和替换，是文本清洗的核心工具。
import requests  # [语法]: 导入第三方库 requests。 [作用]: 用于发送 HTTP 请求，从互联网上下载文件。

# ==========================================
# 1. 定义下载函数 (替代 d2l.DATA_HUB)
# ==========================================
def download_time_machine():
    # [语法]: 变量赋值 (字符串)。
    # [作用]: 定义数据集的下载链接 (URL)。
    url = "http://d2l-data.s3-accelerate.amazonaws.com/timemachine.txt"
    
    # [语法]: 变量赋值 (字符串)。
    # [作用]: 定义文件下载后保存到本地的文件名。
    save_path = "timemachine.txt"

    # [语法]: if 判断 + os.path.exists 函数。
    # [作用]: 检查本地是否已经有了这个文件。如果有，就不重复下载，节省时间。
    if not os.path.exists(save_path):
        print("正在下载 time_machine 数据集...")
        
        # [语法]: requests.get 方法。
        # [作用]: 向服务器发送 GET 请求，获取文件内容。
        response = requests.get(url) 
        
        # [语法]: with open(...) 上下文管理器 + 'wb' 模式。
        # [作用]: 以“二进制写入模式”打开文件。如果你下载的是文本、图片或 zip，用 'wb' 最稳妥。
        with open(save_path, 'wb') as f:
            # [语法]: 文件对象的 write 方法。
            # [作用]: 将下载下来的二进制内容 (response.content) 写入硬盘。
            f.write(response.content) 
            
        print("下载完成！")
    else:
        print("文件已存在，直接读取。")
    
    # [语法]: return 语句。
    # [作用]: 返回保存的文件路径，供后面的函数读取使用。
    return save_path

# ==========================================
# 2. 读取函数 (逻辑保持不变)
# ==========================================
def read_time_machine():
    """将时间机器数据集加载为文本行的列表。"""
    
    # [语法]: 函数调用。
    # [作用]: 调用上面写好的下载函数，确保文件在本地，并获取文件名。
    file_path = download_time_machine()
    
    # [语法]: with open(...) + 'r' 模式 + encoding 参数。
    # [作用]: 以“只读文本模式”打开文件。指定 utf-8 编码是为了防止中文或特殊字符乱码。
    with open(file_path, 'r', encoding='utf-8') as f: 
        # [语法]: readlines() 方法。
        # [作用]: 一次性读取文件所有内容，并按行切割，返回一个列表 (List)。
        lines = f.readlines()
    
    # [语法]: 列表推导式 (List Comprehension) + 链式调用。
    # [作用]: 这是一个高效循环。它遍历 lines 中的每一行 (line)，处理完后生成一个新的列表。
    # 细节拆解:
    # 1. re.sub('[^A-Za-z]+', ' ', line): 正则替换。[^A-Za-z] 表示“非字母的字符”，把它们统统换成空格。
    # 2. .strip(): 去掉字符串开头和结尾的空白符 (换行符 \n, 空格)。
    # 3. .lower(): 把所有大写字母转成小写，统一格式 (如 The -> the)。
    return [ re.sub('[^A-Za-z]+', ' ', line).strip().lower()  for line in lines]

# ==========================================
# 3. 运行测试
# ==========================================
# [语法]: 函数调用。
# [作用]: 执行读取流程，获取清洗后的所有文本行。
lines = read_time_machine()

# [语法]: f-string 格式化字符串 + len() 函数。
# [作用]: 计算列表长度，打印总共有多少行文本。
print(f"总行数: {len(lines)}")

# [语法]: 列表索引 (List Indexing)。
# [作用]: 访问列表的第 1 个元素 (索引 0)。
print("第一行:", lines[0])
print("第二行:", lines[1])

# [语法]: 列表索引。
# [作用]: 访问列表的第 11 个元素 (索引 10)。
print("第十行:", lines[10])

文件已存在，直接读取。
总行数: 3221
第一行: the time machine by h g wells
第二行: 
第十行: twinkled and his usually pale face was flushed and animated the


正则表达式符号详解 [^A-Za-z]+符号语法含义作用[]字符集合匹配方括号中包含的任意一个字符。^取反符号当它出现在方括号的开头时，表示“匹配不在集合中的字符”。A-Za-z字母范围代表所有大写（A-Z）和小写（a-z）的英文字母。+量词（1次或多次）表示前面的模式必须连续出现至少一次。

② 每个文本序列又被拆分成一个标记列表。

In [8]:
def tokenize(lines, token='word'):
    # [语法] 定义一个函数，接受两个参数：lines (必须) 和 token (可选，默认值为 'word')。
    # [作用] 封装分词逻辑，根据用户需求将文本行转换成单词列表或字符列表。
    """
    将文本行列表进行分词处理。
    """
    # 如果令牌类型为 'word'
    if token == 'word':
        # [语法] 列表推导式 (List Comprehension)。遍历 lines 中的每一行 (line)，对每行调用 .split() 方法。
        # [作用] 按“空格”切分每行文本。例如 "Hello World" -> ['Hello', 'World']。这是最基本的英文分词。
        # 注意 line.split() 和 line.split(' ') 完全不一样，左边是"智能模式"
        return [line.split() for line in lines]
    
    # 如果令牌类型为 'char'
    elif token == 'char':
        # [语法] 列表推导式。遍历 lines 中的每一行，使用 list() 构造函数将字符串转换为列表。
        # [作用] 按“字符”切分。例如 "Hi" -> ['H', 'i']。常用于字符级模型或处理中文（因为中文没有空格分隔）。
        return [list(line) for line in lines]
    
    else:
        # [语法] else 分支，处理前面 if 和 elif 都不满足的情况。
        # [作用] 错误处理机制。防止用户输入了不支持的分词类型（如 'sentence'），提示参数错误。
        print('错位：未知令牌类型：' + token)

# ================= 调用部分 =================

# [语法] 调用 tokenize 函数，传入 lines 变量，token 参数省略（使用默认值 'word'），并将结果赋值给 tokens。
# [作用] 执行实际的分词操作。假设 lines 是之前读取的文本列表，现在 tokens 变成了一个二维列表（列表的列表）。
tokens = tokenize(lines)

# [语法] for 循环，range(11) 生成从 0 到 10 的整数序列。
# [作用] 仅遍历前 11 行数据。用于快速预览数据，避免打印整个数据集导致屏幕刷屏。
for i in range(11):
    # [语法] 使用索引 i 访问列表 tokens 中的元素，并打印到控制台。
    # [作用] 打印第 i 行的分词结果。如果原文本有空行，split() 会返回空列表 []。
    print(tokens[i])

['the', 'time', 'machine', 'by', 'h', 'g', 'wells']
[]
[]
[]
[]
['i']
[]
[]
['the', 'time', 'traveller', 'for', 'so', 'it', 'will', 'be', 'convenient', 'to', 'speak', 'of', 'him']
['was', 'expounding', 'a', 'recondite', 'matter', 'to', 'us', 'his', 'grey', 'eyes', 'shone', 'and']
['twinkled', 'and', 'his', 'usually', 'pale', 'face', 'was', 'flushed', 'and', 'animated', 'the']


③ 构建一个字典，通常也叫做词表（vocabulary），用来你将字符串标记映射到从0开始的数字索引中。

In [18]:
import collections  
# [语法] 导入 Python 标准库中的 collections 模块。
# [作用] 为了在后续代码中使用 collections.Counter 类来高效统计词频。
def count_corpus(tokens):
    """
    统计标记的频率。
    
    参数:
        tokens (list): 
            [语法] 列表类型的参数。
            [作用] 待统计的语料，可能是一维 ['hi', 'you'] 或二维 [['hi'], ['you']]。
    """
    # [语法] 
    # 1. len(tokens) == 0: 检查列表是否为空。
    # 2. isinstance(tokens[0], list): 检查列表的第一个元素是否也是列表。
    # [作用] 判断输入数据的格式。如果列表为空，或者列表里装的是列表（说明是二维的句子列表），则需要进行展平操作。
    if len(tokens) == 0 or isinstance(tokens[0], list):
        # [语法] 双层嵌套的列表推导式 (Flatten logic)。
        # 等价于:
        # temp = []
        # for line in tokens:
        #     for token in line:
        #         temp.append(token)
        # [作用] 将二维列表展平为一维列表。例如 [['a', 'b'], ['c']] -> ['a', 'b', 'c']，方便统一统计。
        tokens = [token for line in tokens for token in line]
    
    # [语法] 实例化 collections.Counter 类。
    # [作用] 对列表中的元素进行哈希计数，返回一个 Counter 对象（如 {'a': 1, 'b': 1, 'c': 1}）。
    return collections.Counter(tokens)

class Vocab:
    """文本词表类，用于构建 词<->索引 的映射关系"""
    
    def __init__(self, tokens=None, min_freq=0, reserved_tokens=None):
        """
        初始化词表对象。

        参数详解:
            tokens (list): 
                [语法] 默认为 None 的列表参数。
                [作用] 输入的语料库，可以是一维的单词列表，也可以是二维的句子列表。
            min_freq (int): 
                [语法] 默认为 0 的整数参数。
                [作用] 最小频率阈值。出现次数少于这个数的单词会被丢弃（视为未知词）。
            reserved_tokens (list): 
                [语法] 默认为 None 的列表参数。
                [作用] 需要强制保留的特殊标记，例如 ['<pad>', '<bos>', '<eos>']。
        """
        # [语法] if 条件判断语句，判断 tokens 是否为 None。
        # [作用] 防御性编程。如果用户初始化时什么都没传，将其设为空列表，防止后续 count_corpus 报错。
        if tokens is None:
            tokens = []
        
        # [语法] if 条件判断语句，判断 reserved_tokens 是否为 None。
        # [作用] 如果用户没有指定保留标记，将其初始化为空列表，方便后续进行列表拼接操作。
        if reserved_tokens is None:
            reserved_tokens = []
        
        # [语法] 函数调用，将 tokens 作为参数传入 count_corpus 函数。
        # [作用] 统计语料库中每个 token 出现的次数，返回一个 Counter 对象（类似于字典 {词: 次数}）。
        counter = count_corpus(tokens) 
        
        # [语法] 
        # 1. counter.items(): 获取字典的 (键, 值) 元组列表。
        # 2. key=lambda x: x[1]: 使用匿名函数指定排序依据为元组的第2个元素（即频率）。
        # 3. reverse=True: 指定为降序排序。
        # 4. sorted(): 内置排序函数。
        # [作用] 将词频统计结果按出现频率从高到低排序。这样做是为了后续处理方便，也符合词表的一般构建逻辑（常用词在前）。
        self.token_freqs = sorted(counter.items(), key=lambda x: x[1], reverse=True)
        
        # [语法] 多重赋值语句；列表拼接操作 (+)。
        # [作用] 
        # 1. self.unk = 0: 设定未知词标记 <unk> 的索引永远为 0。
        # 2. uniq_tokens: 初始化唯一词列表，首先加入 <unk> 和用户定义的保留标记（如 <pad>）。
        self.unk, uniq_tokens = 0, ['<unk>'] + reserved_tokens 
        
        # [语法] 
        # 1. uniq_tokens += [...]: 列表的原地拼接。
        # 2. [token for ... if ...]: 带有过滤条件的列表推导式。
        # [作用] 核心过滤逻辑：
        # 遍历排序后的词频列表，将满足 "频率 >= min_freq" 且 "不在保留列表中" 的词添加到 uniq_tokens 中。
        uniq_tokens += [token for token, freq in self.token_freqs
                        if freq >= min_freq and token not in uniq_tokens]
        
        # [语法] 多重赋值；初始化空列表 [] 和空字典 dict()。
        # [作用] 
        # self.idx_to_token: 用于 "索引 -> 词" (解码)。
        # self.token_to_idx: 用于 "词 -> 索引" (编码)。
        self.idx_to_token, self.token_to_idx = [], {}
        
        # [语法] for 循环遍历列表 uniq_tokens。
        # [作用] 遍历最终筛选出的词表，建立双向映射关系。
        for token in uniq_tokens:
            # [语法] 列表的 append 方法。
            # [作用] 将 token 添加到列表中。列表的自然下标（0, 1, 2...）即为该词的索引。
            self.idx_to_token.append(token)
            
            # [语法] 字典赋值操作；len() 获取列表长度。
            # [作用] 记录映射关系：键是 token，值是它在 idx_to_token 中的下标 (即当前长度 - 1)。
            self.token_to_idx[token] = len(self.idx_to_token) - 1
            
    def __len__(self):
        """
        [语法] Python 魔术方法 (Magic Method)，定义当对对象调用 len() 时行为。
        [作用] 返回词表的大小。
        """
        # [语法] len() 函数获取列表长度。
        # [作用] 返回唯一词列表的长度，即词表中词的总数。
        return len(self.idx_to_token)
    
    def __getitem__(self, tokens):
        """
        [语法] Python 魔术方法，定义对象使用索引操作符 object[key] 时的行为。
        [作用] 给定词（或词列表），返回对应的索引（或索引列表）。
        """
        # [语法] not isinstance(..., (list, tuple)): 类型检查，判断 tokens 是否**不是**列表或元组。
        # [作用] 处理单个词的情况。如果传入的是字符串 'apple'，走这个分支。
        if not isinstance(tokens, (list, tuple)):
            # [语法] 字典的 .get(key, default) 方法。
            # [作用] 核心查表：如果词在词表中，返回索引；如果不在，返回 self.unk (即 0)。这是处理 OOV (Out Of Vocabulary) 的关键。
            return self.token_to_idx.get(tokens, self.unk)
        
        # [语法] 列表推导式；递归调用 self.__getitem__。
        # [作用] 处理词列表的情况。如果传入的是 ['apple', 'banana']，遍历列表对每个词递归调用本函数，返回索引列表。
        return [self.__getitem__(token) for token in tokens]
    
    def to_tokens(self, indices):
        """
        [语法] 普通实例方法。
        [作用] 给定索引（或索引列表），返回对应的词（或词列表）。
        """
        # [语法] 类型检查，判断 indices 是否**不是**列表或元组。
        # [作用] 处理单个索引的情况。如果传入是整数 1，走这个分支。
        if not isinstance(indices, (list, tuple)):
            # [语法] 列表索引访问。
            # [作用] 直接返回该索引对应的词。
            return self.idx_to_token[indices]
        
        # [语法] 列表推导式。
        # [作用] 处理索引列表的情况。如果传入 [1, 2]，遍历列表并将每个索引转换为词，返回词列表。
        return [self.idx_to_token[index] for index in indices]
    

④ 构建词汇表

In [19]:
# 创建一个 Vocab 对象，将标记列表 tokens 作为参数传入，用于构建词表
vocab = Vocab(tokens)
# 获取词表中的前 10 个标记及其对应的索引值，并将其转换为列表进行打印输出
print(list(vocab.token_to_idx.items())[:10])

[('<unk>', 0), ('the', 1), ('i', 2), ('and', 3), ('of', 4), ('a', 5), ('to', 6), ('was', 7), ('in', 8), ('that', 9)]


④ 将每一行文本转换成一个数字索引列表。

In [24]:
# 遍历索引列表 [0, 10]
for i in range(10):
    # 打印当前索引 i 处的标记（单词）
    if len(tokens[i]) is not 0 :
        print('word:', tokens[i])
        # 获取当前标记在词表中的索引值
        # 打印当前标记在词表中的索引值（对应的索引值或未知标记索引）
        print('indices:',vocab[tokens[i]])

word: ['the', 'time', 'machine', 'by', 'h', 'g', 'wells']
indices: [1, 19, 50, 40, 2183, 2184, 400]
word: ['i']
indices: [2]
word: ['the', 'time', 'traveller', 'for', 'so', 'it', 'will', 'be', 'convenient', 'to', 'speak', 'of', 'him']
indices: [1, 19, 71, 16, 37, 11, 115, 42, 680, 6, 586, 4, 108]
word: ['was', 'expounding', 'a', 'recondite', 'matter', 'to', 'us', 'his', 'grey', 'eyes', 'shone', 'and']
indices: [7, 1420, 5, 2185, 587, 6, 126, 25, 330, 127, 439, 3]


<>:4: SyntaxWarning: "is not" with a literal. Did you mean "!="?
<>:4: SyntaxWarning: "is not" with a literal. Did you mean "!="?
C:\Users\lenovo\AppData\Local\Temp\ipykernel_37852\2730249240.py:4: SyntaxWarning: "is not" with a literal. Did you mean "!="?
  if len(tokens[i]) is not 0 :


⑤ 将所有内容打包到load_corpus_time_machine函数中。

In [26]:
def load_corpus_time_machine(max_tokens=-1):
    # [语法] 函数定义，带有一个默认参数 max_tokens=-1。
    # [作用] 封装数据加载逻辑。max_tokens 用于限制加载的数据量（-1 表示不限制，加载全部），方便调试小样本。
    """返回时光机器数据集的标记索引列表和词汇表"""
    
    # [语法] 函数调用（假设 read_time_machine 是外部定义好的函数）。
    # [作用] 读取原始文本文件，返回一个包含所有文本行的列表（如 ["Line 1 text", "Line 2 text"]）。
    lines = read_time_machine()
    
    # [语法] 调用之前定义的 tokenize 函数，传入 'char' 参数。
    # [作用] 进行字符级分词。将文本行转换为字符列表的列表（如 [['H', 'e', 'l', 'l', 'o'], ['W', 'o', 'r', 'l', 'd']]）。
    tokens = tokenize(lines, 'char')
    
    # [语法] 实例化 Vocab 类。
    # [作用] 构建词表。统计所有字符的频率，分配唯一的索引（0, 1, 2...），并处理未知字符 <unk>。
    vocab = Vocab(tokens)
    
    # [语法] 嵌套的列表推导式 (Nested List Comprehension)。
    # [逻辑展开] 等价于：
    # corpus = []
    # for line in tokens:      # 遍历每一行
    #     for token in line:   # 遍历行中的每一个字符
    #         corpus.append(vocab[token]) # 将字符查表转换为数字索引
    # [作用] 核心转换步骤：将人类可读的字符（token）全部转换为机器可读的整数索引（index），并将二维列表展平为一维长列表。
    corpus = [vocab[token] for line in tokens for token in line]
    
    # [语法] if 条件判断。
    # [作用] 检查是否需要截断数据。通常用于快速测试模型，避免数据量太大跑不动。
    if max_tokens > 0:
        # [语法] 列表切片 (List Slicing) [:n]。
        # [作用] 保留 corpus 列表的前 max_tokens 个元素，丢弃后面的。
        corpus = corpus[:max_tokens]
        
    # [语法] return 语句返回元组。
    # [作用] 将处理好的“语料库索引序列”和“词表对象”一起返回，供后续模型训练使用。
    return corpus, vocab

# ================= 调用部分 =================

# [语法] 函数调用 + 元组解包 (Tuple Unpacking)。
# [作用] 执行数据加载，将返回的两个结果分别赋值给变量 corpus 和 vocab。
corpus, vocab = load_corpus_time_machine()

# [语法] 创建一个元组，包含两个 len() 函数的返回值。
# [作用] 打印统计信息：
# 1. len(corpus): 语料库的总字符数（即样本总长度）。
# 2. len(vocab): 词表的大小（即有多少个不重复的字符类别）。
len(corpus), len(vocab)

文件已存在，直接读取。


(170580, 28)

In [25]:
a = ' djsaklj sjdkljs sjdskj sjdk skdjdsk sjdksj '
list(a)

[' ',
 'd',
 'j',
 's',
 'a',
 'k',
 'l',
 'j',
 ' ',
 's',
 'j',
 'd',
 'k',
 'l',
 'j',
 's',
 ' ',
 's',
 'j',
 'd',
 's',
 'k',
 'j',
 ' ',
 's',
 'j',
 'd',
 'k',
 ' ',
 's',
 'k',
 'd',
 'j',
 'd',
 's',
 'k',
 ' ',
 's',
 'j',
 'd',
 'k',
 's',
 'j',
 ' ']